In [ ]:
from imutils.video import VideoStream
from imutils import face_utils
import urllib.request
import cv2
import numpy as np
import datetime
import imutils
import time
import dlib
import ssl

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/amit/driver1/facial-landmarks/shape_predictor_68_face_landmarks.dat')

In [ ]:
#To use android camera, use IP Webcam App, copy the url of the image the phone is capturing and paste here
url='http://192.168.43.1:8080/shot.jpg?rnd=655898'

context = ssl._create_unverified_context()

while True:
    imgResp=urllib.request.urlopen(url, context=context)
    imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
    img=cv2.imdecode(imgNp,-1)
    cv2.imwrite('MyImage.jpg', img)
    
    #frame = imutils.resize(img, width=400)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    #print(rects)
    
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        for (name, (j, k)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            (x, y, w, h) = cv2.boundingRect(np.array([shape[j:k]]))
            roi = gray[y:y + h, x:x + w]
            #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            #print(name)
            if(j==36):
                right_eye = roi
            elif(j==42):
                left_eye = roi
                
        #cv2.imwrite('right_eye.jpg', right_eye)
        #print('hi!')
        #right_eye = cv2.imread('right_eye.jpg',0)
        #pupil detection using cdf
        img_flat = right_eye.flatten()

        hist, bin_edges = np.histogram(img_flat, 256, [0,256])
        pixel_prob = hist/img_flat.size

        pixel_cmprob = np.zeros((256,1))
        pixel_cmprob[0] = pixel_prob[0]

        for i in range (1,256):
            pixel_cmprob[i] = pixel_cmprob[i-1] + pixel_prob[i]
    
            new_img = np.zeros((right_eye.shape[0],right_eye.shape[1]))
            for i in range(0, right_eye.shape[0]):
                for j in range(0, right_eye.shape[1]):
                    if pixel_cmprob[right_eye[i,j]] < 0.06:
                        new_img[i,j] = 255
                    else:
                        new_img[i,j] = 0  
                        
        kernel = np.ones((2,2), np.uint8)
        new_img_er1 = cv2.erode(new_img, kernel, iterations=1)
        
        min_arr = np.array([0,0])
        for i in range(0, right_eye.shape[0]):
            for j in range(0, right_eye.shape[1]):
                if(new_img_er1[i,j]==255):
                    if(right_eye[i,j]<right_eye[min_arr[0], min_arr[1]]):
                            min_arr[0] = i
                            min_arr[1] = j
        
        thresh_val1 = np.mean(right_eye[(min_arr[0]-5):(min_arr[0]+5), (min_arr[1]-5):(min_arr[1]+5)])
        
        right_eye_f1 = right_eye
        _,right_eye_f1 = cv2.threshold(right_eye_f1, thresh_val1, 255, cv2.THRESH_BINARY_INV)
        
        min_arr = np.array([0,0])
        for i in range(0, right_eye.shape[0]):
            for j in range(0, right_eye.shape[1]):
                if(right_eye_f1[i,j]==255):
                    if(right_eye[i,j]<right_eye[min_arr[0], min_arr[1]]):
                            min_arr[0] = i
                            min_arr[1] = j
        
        thresh_val2 = np.mean(right_eye[(min_arr[0]-5):(min_arr[0]+5), (min_arr[1]-5):(min_arr[1]+5)])
        
        right_eye_f2 = right_eye
        _,right_eye_f2 = cv2.threshold(right_eye_f2, thresh_val2, 255, cv2.THRESH_BINARY_INV)
        
        min_arr = np.array([0,0])
        for i in range(0, right_eye.shape[0]):
            for j in range(0, right_eye.shape[1]):
                if(right_eye_f2[i,j]==255):
                    if(right_eye[i,j]<right_eye[min_arr[0], min_arr[1]]):
                            min_arr[0] = i
                            min_arr[1] = j
        
        right_eye2 = right_eye
        cv2.circle(right_eye2, (min_arr[1],min_arr[0]), 10, (0,0,255), -1)
        cv2.imshow("right_eye2", right_eye2)

        #cv2.waitKey(0)


    if ord('q')==cv2.waitKey(10):
        exit(0)